# SETUP

In [ ]:
import os
import json
from matplotlib import pyplot as plt
import math
import numpy as np
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import pandas as pd
import seaborn as sns
from PIL import Image
from stable_baselines3 import TD3 as sb3TD3

In [ ]:
# set plot params
plt.rcParams.update({'font.size': 22})
plt.show()
plot_dir = 'Plots/'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

# uci primary colors
uci_blue = (0/255, 62/255, 120/255)
uci_gold = (253/255, 185/255, 19/255)

# uci secondary color palette
uci_light_blue = (106/255, 162/255, 184/255)
uci_light_gray = (197/255, 190/255, 181/255)
uci_dark_blue = (27/255, 61/255, 109/255)
uci_orange = (247/255, 141/255, 45/255)
uci_light_yellow = (247/255, 235/255, 95/255)
uci_dark_gray = (85/255, 87/255, 89/255)
uci_lime_green = (122/255, 184/255, 0/255)

# color blind friendly colors
# https://gist.github.com/thriveth/8560036
color_blinds = {
    'blue':   [55/255,  126/255, 184/255],  #377eb8 
    'orange': [255/255, 127/255, 0/255],    #ff7f00
    'green':  [77/255,  175/255, 74/255],   #4daf4a
    'pink':   [247/255, 129/255, 191/255],  #f781bf
    'brown':  [166/255, 86/255,  40/255],   #a65628
    'purple': [152/255, 78/255,  163/255],  #984ea3
    'gray':   [153/255, 153/255, 153/255],  #999999
    'red':    [228/255, 26/255,  28/255],   #e41a1c
    'yellow': [222/255, 222/255, 0/255]     #dede00
} 
color_blinds_list = [color_blinds[color] for color in color_blinds]
unique_markers = ['o', 's', 'p', '*', 'X', 'd']

object_color = uci_blue

# BINVOX

In [ ]:
import binvox as bv
binvox_path = 'map_voxels.binvox'
voxels = bv.Binvox.read(binvox_path, 'dense')
voxels_data = voxels.data
voxels_scale = voxels.scale 
voxels_trans = voxels.translate
voxels_res = (np.absolute(voxels_trans)) * 2 * voxels_scale

In [ ]:
_global_plt_patches = None

def plot_map(axis, object_color='gray'):
    global _global_plt_patches
    if _global_plt_patches is None:
        origin = voxels_data[int(voxels_data.shape[1]/2), int(voxels_data.shape[0]/2), :]
        floor_dim = max([i for i, x in enumerate(origin) if x])
        print(floor_dim)
        scale = voxels_scale * 100
        shift = voxels_trans
        _global_plt_patches = []
        for x in range(voxels_data.shape[0]):
            for y in range(voxels_data.shape[1]):
                for z in range(voxels_data.shape[2]):
                    if voxels_data[x, y, z] and z > floor_dim:
                        #x_loc = x - shift - scale/2 # align to left for plt.rect
                        #y_loc = y - shift - scale/2 # align to bottom for plt.rect
                        #patch = patches.Rectangle((x_loc, y_loc), scale, scale, color = object_color)
                        patch = patches.Rectangle((x+shift[0], y+shift[1]), 1, 1, color = object_color)
                        _global_plt_patches.append(patch)
                        break
    # add list of patches (much quicker than iteratively drawing)
    map_stuff = PatchCollection(_global_plt_patches, match_original=True)
   # axis.gca().add_collection(map_stuff)
    axis.add_collection(map_stuff)

In [ ]:
def plot_spawns_goals(
    drone_radius = 0.5,
    goal_tolerance = 4,
    goal_distance = 100,
    spawns = [],
    goals = [],
    spawn_color = color_blinds['red'],
    goal_color = color_blinds['orange'],
    goal_wedge = True,
):
    # patch stuff
    spawn_patches = []
    goal_patches = []
    for idx in range(len(spawns)):
        spawn = spawns[idx]
        goal = goals[idx]
        spawn_patch = patches.Rectangle(
            (spawn[0], spawn[1]), 
            drone_radius, 
            drone_radius, 
            color=spawn_color,
        )
        spawn_patches.append(spawn_patch)
        if goal_wedge:
            radius = math.sqrt(goal[0]**2 + goal[1]**2)
            goal_patch = patches.Wedge(
                (spawn[0], spawn[1]), 
                goal_distance+goal_tolerance, 
                goal[0], goal[1], 
                width=2*goal_tolerance,
                color=goal_color,
            )
        else:
            goal_patch = patches.Circle(
                (goal[0], goal[1]), 
                goal_tolerance, 
                color=goal_color,
            )
        goal_patches.append(goal_patch)
    # add list of patches (much quicker than iteratively drawing)
    spawn_collection = PatchCollection(spawn_patches, match_original=True)
    goal_collection = PatchCollection(goal_patches, match_original=True)
    plt.gca().add_collection(goal_collection)
    plt.gca().add_collection(spawn_collection)

In [ ]:
def plot_environment(name='Learning Environment (to scale)'):
    fig = plt.figure()
    fig.set_size_inches((6, 6))
    plt.title(name)
    plt.xlabel('y [meters]')
    plt.ylabel('x [meters]')
    plt.xlim(-120, 120)
    plt.ylim(-120, 120)
    # custom legend hack
    plt.scatter(-999, -999, marker='s', color=object_color)
    plt.scatter(-999, -999, marker='s', color=train_spawn_color)
    plt.scatter(-999, -999, color=train_goal_color)
    #plt.scatter(-999, -999, color=eval_spawn_color)
    #plt.scatter(-999, -999, color=eval_goal_color)
    plt.legend([
        'Objects', 
        'Spawn', 
        'Goal', 
        #'Eval Spawn', 
        #'Eval Goal'
    ],
               loc='center left', 
               bbox_to_anchor=(1, 0.5),)

# EVALUATIONS

In [ ]:
# json files output with all string key names
# process so that the evaluation dictionary structure is such:
    # episode # - int
        # step # - int
            # state - dictionary of misc key, value pairs for that state
def process_episodes(json_evaluation):
    nEpisodes = len(json_evaluation)
    episodes = [None] * nEpisodes
    episode_idx = 0
    for episode_str in json_evaluation:
        if 'episode_' not in episode_str:
            continue
        json_episode = json_evaluation[episode_str]
        nSteps = len(json_episode)
        states = [None] * nSteps
        for step_str in json_episode:
            step_num = int(step_str.split('_')[1])
            state = json_episode[step_str]
            states[step_num] = state
        episodes[episode_idx] = states
        episode_idx += 1
    return episodes
def read_evaluations(evaluation_folder):
    evaluation_files = [file for file in os.listdir(evaluation_folder) if 'states' in file]
    nEvaluations = len(evaluation_files)
    evaluations = [None] * nEvaluations
    for evaluation_file in evaluation_files:
        if '.json' not in evaluation_file:
            continue
        epoch = int(evaluation_file.split('.')[0].split('_')[-1])
        print(evaluation_file, epoch)
        json_evaluation = json.load(open(evaluation_folder + evaluation_file, 'r'))
        episodes = process_episodes(json_evaluation)
        evaluations[epoch] = episodes
    return evaluations
# architecture for evaluations:
# evaluations - list of episodes (indexed of evaluation number) - 0 idx is first evaluation
    # episodes - list of states (indexed by step number)
        # states - dict of (key, value) pairs for state at all_evaluations[instance][evaluation][episode][step]

In [ ]:
# read evaluations from each instance
# each instance is a sub folder from parent with which this eval notebook is in
instances = [
    'EvaluateEnvironment',
    'TrainEnvironment',
    ]
all_evaluations = {}
for instance in instances:
    all_evaluations[instance] = read_evaluations(instance + '/')

In [ ]:
# read info from config file
#config_file = json.load(open('configuration.json', 'r'))
gamma = 0.99#config_file['components']['Model']['gamma']

In [ ]:
# process evaluations
final_variables = {instance:{} for instance in instances}
for instance in instances:
    for set_num, episodes in enumerate(all_evaluations[instance]):
        for episode, states in enumerate(episodes):
            for step, state in enumerate(states):
                if step == 0:
                    if 'goal_distance' not in final_variables[instance]:
                        final_variables[instance]['goal_distance'] = {}
                    if episode not in final_variables[instance]['goal_distance']:
                        final_variables[instance]['goal_distance'][episode] = []
                    drone_position = np.array(state['drone_position'])
                    goal_position = np.array(state['goal_position'])
                    goal_distance = np.linalg.norm(goal_position - drone_position)
                    final_variables[instance]['goal_distance'][episode].append(goal_distance)
                    continue
            final_state = states[-1]
            for variable in final_state:
                if variable not in final_variables[instance]:
                    final_variables[instance][variable] = {}
                if episode not in final_variables[instance][variable]:
                    final_variables[instance][variable][episode] = []
                final_variables[instance][variable][episode].append(final_state[variable])

# architecture of final_variables:
# instance - str name
    # variable - str name
        # episode - int number
            # final values - list of final values with increased # of training evaluations 

In [ ]:
# PLOT RESULTS VERSUS TRAINING TIME
results_instances = [
    'EvaluateEnvironment', 
    'TrainEnvironment', 
]
# select which state variables to fetch
numerical_variables = [
    #'nSteps', 
    'total_reward', 
    'goal_distance',
]
string_variables = [
    #'transcribed_action', 
    'termination_reason',
]
all_variables = numerical_variables + string_variables

shorten = {
    'termination_reason':'termination',
}

# take mean values
for variable in all_variables:
    for instance in results_instances:
        if variable not in shorten:
            shorten[variable] = variable
        plot_name = instance + '_' + variable
        
        values = {}
        for episode in final_variables[instance][variable]:
            for evaluation in range(len(final_variables[instance][variable][episode])):
                if evaluation not in values:
                    values[evaluation] = []
                values[evaluation].append(final_variables[instance][variable][episode][evaluation])

        if variable in numerical_variables:
            aggregates = []
            nEpisodes = len(final_variables[instance][variable])
            for evaluation in values:
                aggregates.append(sum(values[evaluation])/len(values[evaluation]))
            plt.title(instance)
            plt.xlabel('Epoch')
            plt.ylabel(variable)
            plt.plot(aggregates)
            plt.tight_layout()
            fig = plt.gcf()
            fig.set_size_inches(8, 4)
            plt.savefig(plot_dir + plot_name + '.png')
            plt.show()

        if variable in string_variables:
            fig = plt.gcf()
            fig.set_size_inches(4, 4)
            aggregates = {}
            for evaluation in values:
                for name in values[evaluation]:
                    if name not in aggregates:
                        aggregates[name] = {}
                    if evaluation not in aggregates[name]:
                        aggregates[name][evaluation] = 0
                    aggregates[name][evaluation] += 1   
            df_data = {'count':[], shorten[variable]:[], 'epoch':[]}
            for name in aggregates:
                for evaluation in values:
                    if evaluation not in aggregates[name]:
                        aggregates[name][evaluation] = 0
                    count = aggregates[name][evaluation]
                    df_data['count'].append(count)
                    df_data[shorten[variable]].append(name[0])
                    df_data['epoch'].append(evaluation)
            df = pd.DataFrame(df_data)
            heatmap_data = pd.pivot_table(df, values='count', index=[shorten[variable]], columns='epoch')
            sns.heatmap(heatmap_data, cbar=False)
            plt.title(instance)
            plt.tight_layout()
            plt.savefig(plot_dir + plot_name + '.png')
            plt.show()

In [ ]:
res = True
# PLOT SUCCESS PATHS
path_instances = [
    'EvaluateEnvironment', 
    #'TrainEnvironment', 
]
for instance in path_instances:
    evaluations = all_evaluations[instance]
    for set_num, episodes in enumerate(evaluations):
        paths = []
        reward_paths = []
        rho_paths = []
        pf_paths = []
        pd_paths = []
        Q_paths = []
        goals = []
        spawns = []
        termination_reasons = []
        episode_numbers = []
        # do we plot this set? (change as needed)
        # you can just plot everything but its alot to chew
        view_set = True
        for episode, states in enumerate(episodes):
            # get final vaules
            final_state = episodes[episode][-1]
            termination_result = final_state['termination_result']
            #if 'success' not in termination_result:
            #    view_set = False
            #    break
        if not view_set:
            continue
        for episode, states in enumerate(episodes):
            # get final vaules
            final_state = episodes[episode][-1]
            termination_reason = final_state['termination_reason']
            # build paths
            path = []
            reward_path = []
            rho_path = []
            pf_path = []
            pd_path = []
            Q_path = []
            episode_numbers.append(episode)
            # get initial value
            init_state = states[0]
            spawn = init_state['drone_position']
            goal = init_state['goal_position']
            # get intermediate values
            for step, state in enumerate(states):
                if step == 0: continue
                state = episodes[episode][step]
                drone_position = state['drone_position']
                path.append(drone_position)
                reward = state['total_reward']
                reward_path.append(reward)
                rho = state['transcribed_action']['slim']
                rho_path.append(rho)
                pf = state['transcribed_action']['FlattenedDepthResolution']
                pf_path.append(pf)
                pd = state['transcribed_action']['FlattenedDepthResolution2']
                pd_path.append(pd)
                # get future values from this state
                for step2, state2 in enumerate(states[step+1:]):
                    if step == 0: continue
                    reward += (gamma**(step2-step)) * state2['total_reward']
                Q_path.append(reward)
            reward_paths.append(reward_path)
            rho_paths.append(rho_path)
            pf_paths.append(pf_path)
            pd_paths.append(pd_path)
            termination_reasons.append(termination_reason)
            spawns.append(spawn)
            goals.append(goal)
            paths.append(path)
            Q_paths.append(Q_path)
            # plot every 6 episodes 
            if episode == len(episodes)-1 or (episode > 0 and episode%6 == 0):
                print(set_num)
                # subplots
                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
                fig.set_size_inches(16, 8)
                ax1 = plt.subplot2grid((2, 2), (0, 0), rowspan=2)
                ax2 = plt.subplot2grid((2, 2), (0, 1), colspan=1)
                ax3 = plt.subplot2grid((2, 2), (1, 1), colspan=1)
                # PLOT DRONE PATH
                ax1.set_title(f'Drone Path (to scale)')#' Epoch #{set_num}')
                ax1.set_xlabel('y [meters]')
                ax1.set_ylabel('x [meters]')
                ax1.set_xlim(voxels_trans[0], -1*voxels_trans[0])
                ax1.set_ylim(voxels_trans[1], -1*voxels_trans[1])
                # show objects on map from binvox
                plot_map(ax1)
                # legend hack
                ax1.scatter(-999, -999, marker='s', color=object_color) # off map just for legend
                legend = [
                #    'Objects'
                ]
                for idx in range(len(termination_reasons)):
                    color = color_blinds_list[idx]
                    radius = 64 * spawns[idx][2]/-4
                    #ax1.scatter(spawns[idx][1], spawns[idx][0], marker='x', color=color) # spawn location
                    ax1.scatter(spawns[idx][1], spawns[idx][0], s=radius, color=color, marker = unique_markers[idx])
                    termination_reason = termination_reasons[idx]
                    legend.append('E' + str(episode_numbers[idx]) + ':' + termination_reason)
                plt_patches = []
                for idx in range(len(paths)):
                    color = color_blinds_list[idx]
                    # plot goal
                    obj = plt.Circle((goals[idx][1], goals[idx][0]), 4, color=color, alpha=0.4)
                    plt_patches.append(obj)
                    # plot points
                    for point in paths[idx]:
                        # using generic quad copter size for point sizes (set this to your drone value)
                        radius = 64 * point[2]/-4
                        ax1.scatter(point[1], point[0], s=radius, color=color, marker = unique_markers[idx])
                        #point = plt.Circle((point[1], point[0]), radius, color=color)
                        #plt_patches.append(point)
                    # plot end point
                    #ax1.scatter(paths[idx][-1][1], paths[idx][-1][0], marker='*', s=64, color=color) # end location
                map_stuff = PatchCollection(plt_patches, match_original=True)
                ax1.add_collection(map_stuff)
                plt.tight_layout()
                # PLOT REWARDS
                if not res:
                    ax2.set_title('Slimmable Factor Path')
                    ax2.set_ylabel(r'$rho$')
                    for idx in range(len(rho_paths)):
                        ax2.scatter([i+1.1+.1*idx for i in range(len(rho_paths[idx]))], rho_paths[idx]
                                    , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                    ax2.set_ylim([-.1, 1.1])
                else:
                    ax2.set_title('Forward Facing Sensor')
                    ax2.set_ylabel(r'$p_f$')
                    for idx in range(len(pf_paths)):
                        ax2.scatter([i+1.1+.1*idx for i in range(len(pf_paths[idx]))], pf_paths[idx]
                                    , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                    ax2.set_ylim([-.1, 3.1])
                max_step = 0
                for idx in range(len(rho_paths)):
                    max_step = max(max_step, len(rho_paths[idx]))
                xticks = [i+1.5 for i in range(max_step)]
                labels = [str(i+1) for i in range(max_step)]
                #ax2.set_xticks(xticks)
                #ax2.grid(axis='x')
                for i in range(max_step):
                    ax2.axvline(x = i+1, color = 'gray', ls=':')
                ax2.set_xticks(xticks, labels, rotation = 45)
                ax2.set_xlim(1, max_step+1)
                #ax2.set_xticklabels(ax2.get_xticks(), rotation = 45)
                '''
                ax2.set_title('Reward Path')
                #ax2.set_xlabel('Step')
                ax2.set_ylabel('Immediate Reward')
                for idx in range(len(reward_paths)):
                    color = color_blinds_list[idx]
                    ax2.plot([i+1 for i in range(len(reward_paths[idx]))], reward_paths[idx], color=color, marker='.')
                '''
                # PLOT Q-VALUES
                ax3.set_xlabel('Step')
                if not res:
                    ax3.set_title('Q-value Path')
                    ax3.set_ylabel('Accumlated Reward')
                    for idx in range(len(Q_paths)):
                        ax3.scatter([i+1.1+.1*idx for i in range(len(Q_paths[idx]))], Q_paths[idx]
                                    , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                else:
                    ax3.set_title('Downward Facing Sensor')
                    ax3.set_ylabel(r'$p_d$')
                    for idx in range(len(pd_paths)):
                        ax3.scatter([i+1.1+.1*idx for i in range(len(pd_paths[idx]))], pd_paths[idx]
                                    , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                    ax3.set_ylim([-.1, 3.1])
                ax3.set_xticks(xticks, labels, rotation = 45)
                for i in range(max_step):
                    ax3.axvline(x = i+1, color = 'gray', ls=':')
                ax3.set_xticks(xticks, labels, rotation = 45)
                ax3.set_xlim(1, max_step+1)
                # plot legend
                #ax1.legend(legend, loc='lower left', bbox_to_anchor=(0., 0.), fancybox=True, shadow=True, ncol=6)
                # Show the graph
                plot_name = instance + '_best_path'
                plt.savefig(plot_dir + plot_name + '.png')
                plt.show()
                # reset path arrays
                paths = []
                reward_paths = []
                Q_paths = []
                goals = []
                spawns = []
                termination_reasons = []
                episode_numbers = []

In [ ]:
# PLOT SUCCESS PATHS
path_instances = [
    'EvaluateEnvironment', 
    #'TrainEnvironment', 
]
for instance in path_instances:
    evaluations = all_evaluations[instance]
    for set_num, episodes in enumerate(evaluations):
        paths = []
        reward_paths = []
        rho_paths = []
        pf_paths = []
        pd_paths = []
        Q_paths = []
        goals = []
        spawns = []
        termination_reasons = []
        episode_numbers = []
        # do we plot this set? (change as needed)
        # you can just plot everything but its alot to chew
        view_set = True
        for episode, states in enumerate(episodes):
            # get final vaules
            final_state = episodes[episode][-1]
            termination_result = final_state['termination_result']
            #if 'success' not in termination_result:
            #    view_set = False
            #    break
        if not view_set:
            continue
        for episode, states in enumerate(episodes):
            # get final vaules
            final_state = episodes[episode][-1]
            termination_reason = final_state['termination_reason']
            # build paths
            path = []
            reward_path = []
            rho_path = []
            pf_path = []
            pd_path = []
            Q_path = []
            episode_numbers.append(episode)
            # get initial value
            init_state = states[0]
            spawn = init_state['drone_position']
            goal = init_state['goal_position']
            # get intermediate values
            for step, state in enumerate(states):
                if step == 0: continue
                state = episodes[episode][step]
                drone_position = state['drone_position']
                path.append(drone_position)
                reward = state['total_reward']
                reward_path.append(reward)
                rho = state['transcribed_action']['slim']
                rho_path.append(rho)
                pf = state['transcribed_action']['FlattenedDepthResolution']
                pf_path.append(pf)
                pd = state['transcribed_action']['FlattenedDepthResolution2']
                pd_path.append(pd)
                # get future values from this state
                for step2, state2 in enumerate(states[step+1:]):
                    if step == 0: continue
                    reward += (gamma**(step2-step)) * state2['total_reward']
                Q_path.append(reward)
            reward_paths.append(reward_path)
            rho_paths.append(rho_path)
            pf_paths.append(pf_path)
            pd_paths.append(pd_path)
            termination_reasons.append(termination_reason)
            spawns.append(spawn)
            goals.append(goal)
            paths.append(path)
            Q_paths.append(Q_path)
            # plot every 6 episodes 
            if episode == len(episodes)-1 or (episode > 0 and episode%6 == 0):
                print(set_num)
                # subplots
                fig, axs = plt.subplots(nrows=3, ncols=2)
                fig.set_size_inches(16, 12)
                ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)
                ax2 = plt.subplot2grid((3, 2), (0, 1), colspan=1)
                ax3 = plt.subplot2grid((3, 2), (1, 1), colspan=1)
                ax4 = plt.subplot2grid((3, 2), (2, 1), colspan=1)
                ax5 = plt.subplot2grid((3, 2), (2, 0), colspan=1)
                # PLOT DRONE PATH
                ax1.set_title(f'Drone Path (to scale)')#' Epoch #{set_num}')
                ax1.set_xlabel('y [meters]')
                ax1.set_ylabel('x [meters]')
                ax1.set_xlim(voxels_trans[0], -1*voxels_trans[0])
                ax1.set_ylim(voxels_trans[1], -1*voxels_trans[1])
                # show objects on map from binvox
                plot_map(ax1)
                # legend hack
                ax1.scatter(-999, -999, marker='s', color=object_color) # off map just for legend
                legend = [
                #    'Objects'
                ]
                for idx in range(len(termination_reasons)):
                    color = color_blinds_list[idx]
                    radius = 64 * spawns[idx][2]/-4
                    #ax1.scatter(spawns[idx][1], spawns[idx][0], marker='x', color=color) # spawn location
                    ax1.scatter(spawns[idx][1], spawns[idx][0], s=radius, color=color, marker = unique_markers[idx])
                    termination_reason = termination_reasons[idx]
                    legend.append('E' + str(episode_numbers[idx]) + ':' + termination_reason)
                plt_patches = []
                for idx in range(len(paths)):
                    color = color_blinds_list[idx]
                    # plot goal
                    obj = plt.Circle((goals[idx][1], goals[idx][0]), 4, color=color, alpha=0.4)
                    plt_patches.append(obj)
                    # plot points
                    for point in paths[idx]:
                        # using generic quad copter size for point sizes (set this to your drone value)
                        radius = 64 * point[2]/-4
                        ax1.scatter(point[1], point[0], s=radius, color=color, marker = unique_markers[idx])
                        #point = plt.Circle((point[1], point[0]), radius, color=color)
                        #plt_patches.append(point)
                    # plot end point
                    #ax1.scatter(paths[idx][-1][1], paths[idx][-1][0], marker='*', s=64, color=color) # end location
                map_stuff = PatchCollection(plt_patches, match_original=True)
                ax1.add_collection(map_stuff)
                # max steps and ticks
                plt.tight_layout()
                max_step = 0
                for idx in range(len(rho_paths)):
                    max_step = max(max_step, len(rho_paths[idx]))
                xticks = [i+1.5 for i in range(max_step)]
                labels = [str(i+1) for i in range(max_step)]
                # Slimmable Factor Path
                ax2.set_title('Slimmable Factor Path')
                ax2.set_ylabel(r'$rho$')
                for idx in range(len(rho_paths)):
                    ax2.scatter([i+1.1+.1*idx for i in range(len(rho_paths[idx]))], rho_paths[idx]
                                , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                ax2.set_ylim([-.1, 1.1])
                # Forward Facing Sensor
                ax3.set_title('Forward Facing Sensor')
                ax3.set_ylabel(r'$p_f$')
                for idx in range(len(pf_paths)):
                    ax3.scatter([i+1.1+.1*idx for i in range(len(pf_paths[idx]))], pf_paths[idx]
                                , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                ax3.set_ylim([-.1, 3.1])
                # Downward Facing Sensor
                ax4.set_xlabel('Step')
                ax4.set_title('Downward Facing Sensor')
                ax4.set_ylabel(r'$p_d$')
                for idx in range(len(pd_paths)):
                    ax4.scatter([i+1.1+.1*idx for i in range(len(pd_paths[idx]))], pd_paths[idx]
                                , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                ax4.set_ylim([-.1, 3.1])
                # Q-value Path
                ax5.set_xlabel('Step')
                ax5.set_title('Q-value Path')
                ax5.set_ylabel('Accumlated Reward')
                for idx in range(len(Q_paths)):
                    ax5.scatter([i+1.1+.1*idx for i in range(len(Q_paths[idx]))], Q_paths[idx]
                                , color=color_blinds_list[idx], marker=unique_markers[idx], s=64)
                # vert lines
                for i in range(max_step):
                    ax2.axvline(x = i+1, color = 'gray', ls=':')
                ax2.set_xticks(xticks, labels, rotation = 45)
                ax2.set_xlim(1, max_step+1)
                for i in range(max_step):
                    ax3.axvline(x = i+1, color = 'gray', ls=':')
                ax3.set_xticks(xticks, labels, rotation = 45)
                ax3.set_xlim(1, max_step+1)
                for i in range(max_step):
                    ax4.axvline(x = i+1, color = 'gray', ls=':')
                ax4.set_xticks(xticks, labels, rotation = 45)
                ax4.set_xlim(1, max_step+1)
                for i in range(max_step):
                    ax5.axvline(x = i+1, color = 'gray', ls=':')
                ax5.set_xticks(xticks, labels, rotation = 45)
                ax5.set_xlim(1, max_step+1)
               
            
                # Show the graph
                plot_name = instance + '_best_path'
                plt.savefig(plot_dir + plot_name + '.png')
                plt.show()
                # reset path arrays
                paths = []
                reward_paths = []
                Q_paths = []
                goals = []
                spawns = []
                termination_reasons = []
                episode_numbers = []

# SUMMARY FIGURE

In [ ]:
img1 = np.asarray(Image.open(plot_dir + 'EvaluateEnvironment_goal_distance.png'))
img2 = np.asarray(Image.open(plot_dir + 'EvaluateEnvironment_best_path.png'))
img3 = np.asarray(Image.open(plot_dir + 'EvaluateEnvironment_termination_reason.png'))
img4 = np.asarray(Image.open(plot_dir + 'TrainEnvironment_termination_reason.png'))

gshape = [img2.shape[0], img1.shape[1] + img2.shape[1], 4]
gimg = np.zeros(gshape, dtype=np.uint8)
gimg[:img1.shape[0],:img1.shape[1],:] = img1
gimg[:,img1.shape[1]:,:] = img2
gimg[img1.shape[0]:,:img3.shape[1],:] = img3
gimg[img1.shape[0]:,img3.shape[1]:img1.shape[1],:] = img4

plt.imshow(gimg)
plt.axis('off')
fig = plt.gcf()
fig.set_size_inches(24, 8)
plt.tight_layout()
plt.show()

# SUPPLEMENTAL SCRIPTS

In [ ]:
# process evaluations
instance = 'EvaluateEnvironment'
#set_num = 260
mean_levels = []
for set_num, episodes in enumerate(all_evaluations[instance]):
    if set_num > 142:
        break
    episodes = all_evaluations[instance][set_num]
    resolution_levels = []
    for episode, states in enumerate(episodes):
        for step, state in enumerate(states):
            if step == 0:
                drone_position = np.array(state['drone_position'])
                goal_position = np.array(state['goal_position'])
                goal_distance = np.linalg.norm(goal_position - drone_position)
                continue
            else:
                resolution_level = int(4*state['rl_output'][0])
                resolution_levels.append(resolution_level)
    mean_level = np.mean(resolution_levels)
    mean_levels.append(mean_level)
plt.plot(mean_levels)
plt.title('Blocks Environment')
plt.xlabel('Epoch')
plt.ylabel('Mean Resolution Level')
plt.show()
# architecture of final_variables:
# instance - str name
    # variable - str name
        # episode - int number
            # final values - list of final values with increased # of training evaluations 

In [ ]:
# VIEW ALL PATHS (THIS CAN BE ALOT)
# # plot final paths (run necessary binvox stuff at bottom of notebook to use plot_map)
for instance in instances:
    evaluations = all_evaluations[instance]
    if 'train' in instance:
        evaluations = [evaluations[-1]]
    for set_num, episodes in enumerate(evaluations):
        if set_num > 100:
            continue
        paths = []
        reward_paths = []
        Q_paths = []
        goals = []
        spawns = []
        termination_reasons = []
        episode_numbers = []
        for episode, states in enumerate(episodes):
            path = []
            reward_path = []
            Q_path = []
            episode_numbers.append(episode)
            # get initial value
            init_state = states[0]
            spawn = init_state['drone_position']
            spawns.append(spawn)
            goal = init_state['goal_position']
            goals.append(goal)
            # get intermediate values
            for step, state in enumerate(states):
                if step == 0: continue
                state = episodes[episode][step]
                drone_position = state['drone_position']
                path.append(drone_position)
                reward = state['total_reward']
                reward_path.append(reward)
                # get future values from this state
                for step2, state2 in enumerate(states[step+1:]):
                    if step == 0: continue
                    reward += (gamma**(step2-step)) * state2['total_reward']
                Q_path.append(reward)
            paths.append(path)
            reward_paths.append(reward_path)
            Q_paths.append(Q_path)
            # get final vaules
            final_state = episodes[episode][-1]
            termination_reason = final_state['termination_reason']
            termination_reasons.append(termination_reason)
            # plot every 8 episodes 
            if episode == len(episodes)-1 or (episode > 0 and episode%8 == 0):
                # subplots
                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2)
                if 'train' in instance:
                    fig.suptitle(instance + ' Final Set')
                else:
                    fig.suptitle(instance + ' Set #' + str(set_num))
                fig.set_figheight(8)
                fig.set_figwidth(16)
                ax1 = plt.subplot2grid((2, 2), (0, 0), rowspan=2)
                ax2 = plt.subplot2grid((2, 2), (0, 1), colspan=1)
                ax3 = plt.subplot2grid((2, 2), (1, 1), colspan=1)
                # PLOT DRONE PATH
                ax1.set_title('Drone Path (to scale)')
                ax1.set_xlabel('y [meters]')
                ax1.set_ylabel('x [meters]')
                ax1.set_xlim(-100, 100)
                ax1.set_ylim(-100, 100)
                # show objects on map from binvox
                plot_map(ax1)
                # legend hack
                ax1.scatter(-999, -999, marker='s', color=object_color) # off map just for legend
                legend = [
                #    'Objects'
                ]
                for idx in range(len(termination_reasons)):
                    color = color_blinds_list[idx]
                    ax1.scatter(spawns[idx][1], spawns[idx][0], marker='x', color=color) # spawn location
                    termination_reason = termination_reasons[idx]
                    legend.append('E' + str(episode_numbers[idx]) + ':' + termination_reason)
                plt_patches = []
                for idx in range(len(paths)):
                    color = color_blinds_list[idx]
                    # plot goal
                    obj = plt.Circle((goals[idx][1], goals[idx][0]), 4, color=color, alpha=0.4)
                    plt_patches.append(obj)
                    # plot points
                    for point in paths[idx]:
                        # using generic quad copter size for point sizes (set this to your drone value)
                        radius = 0.5
                        point = plt.Circle((point[1], point[0]), radius, color=color)
                        plt_patches.append(point)
                    # plot end point
                    ax1.scatter(paths[idx][-1][1], paths[idx][-1][0], marker='*', s=64, color=color) # end location
                map_stuff = PatchCollection(plt_patches, match_original=True)
                ax1.add_collection(map_stuff)
                # PLOT REWARDS
                ax2.set_title('Reward Path')
                #ax2.set_xlabel('Step')
                ax2.set_ylabel('Immediate Reward')
                for idx in range(len(reward_paths)):
                    color = color_blinds_list[idx]
                    ax2.plot([i+1 for i in range(len(reward_paths[idx]))], reward_paths[idx], color=color, marker='.')
                # PLOT Q-VALUES
                ax3.set_title('Q-value Path')
                ax3.set_xlabel('Step')
                ax3.set_ylabel('Accumlated Reward')
                for idx in range(len(Q_paths)):
                    color = color_blinds_list[idx]
                    ax3.plot([i+1 for i in range(len(Q_paths[idx]))], Q_paths[idx], color=color, marker='.')
                # plot legend
                ax2.legend(legend, loc='center left', bbox_to_anchor=(1, 0.5))
                # Show the graph
                plt.show()
                # reset path arrays
                paths = []
                reward_paths = []
                Q_paths = []
                goals = []
                spawns = []
                termination_reasons = []
                episode_numbers = []

In [ ]:
# MOVE HISTORY
# view moves for each evaluation for each episode
# also keeps track of an AirSim bug that places drone below floor
bugs = []
read_instances =[
    'EvaluateEnvironment', 
    #'TrainEnvironment', 
]
for instance in read_instances:
    evaluations = all_evaluations[instance]
    for set_num, episodes in enumerate(evaluations):
        for episode, states in enumerate(episodes):
            for step, state in enumerate(states):
                if state['drone_position'][2] > 0:
                    bugs.append((instance, set_num, episode, state['drone_position']))
            if 'train' not in instance or set_num == len(evaluations)-1:
                if 'train' in instance:
                    print('Instance', instance, 'Final Set', 'Epsode', episode)
                else:
                    print('Instance', instance, 'Set#', set_num, 'Epsode', episode)
                for step, state in enumerate(states):
                    if step == 0: continue
                    action = state['rl_output']
                    marker = np.round(action,2)
                    if 'Right' in action:
                        marker = 'R'
                    if 'Left' in action:
                        marker = 'L'
                    if 'Forward' in action:
                        marker = 'F'
                    if 'Up' in action:
                        marker = 'U'
                    if 'Down' in action:
                        marker = 'D'
                    print(marker, end=' ')
                termination_reason = states[-1]['termination_reason']
                print(termination_reason)
bugs

In [ ]:
# view specific observations

# read observations
directory = 'TrainEnvironment/'
files = [file for file in os.listdir(directory) if 'observations' in file]
observations = {}
for file in files:
    observation_set = np.load(directory + file, allow_pickle=True)
    observations.update(observation_set)

In [ ]:
# process evaluations
instance = 'TrainEnvironment'
max_res = 4
last_rl0 = max_res
last_rl1 = max_res
for set_num, episodes in enumerate(all_evaluations[instance]):
    for episode, states in enumerate(episodes):
        for step, state in enumerate(states):
            if step == 0:
                last_rl0 = max_res
                last_rl1 = max_res
                l0 = 's'
                l1 = 's'
            if step > 0:
                obs_name = state['observation_name']
                obs = observations[obs_name]
                print(len(obs))
                np.set_printoptions(precision=2)
                print(f'STEP:{step}')
                print(f'Forward Facing Depths, res level = {last_rl0} {l0}')
                print(obs[6:87].reshape(9,9))
                print(f'Forward Facing Depths, res level = {last_rl1} {l1}')
                print(obs[87:87+81].reshape(9,9))
                print()
                last_rl0 = int((max_res+1)*state['rl_output'][0])
                l0 = state['rl_output'][0]
                last_rl1 = int((max_res+1)*state['rl_output'][1])
                l1 = state['rl_output'][1]
        x = input()

In [ ]:
obs_name = 'V165061_V165062_V165063_V165065_V165066_V165055_V165056_V165057_V165059_V165060_V165049_V165050_V165051_V165053_V165054'
obs = observations[obs_name]
nTimesteps = 4
vLength = int(len(obs) / nTimesteps)
for n in range(nTimesteps):
    print(f'At timestep {n}:')
    print(obs[vLength*n:vLength*n+vLength])

In [ ]:
# SLIDESHOW
import cv2
# use this code to make a slideshow from given observations
# will require some small editing for your needs
# will plot various time steps along with rewards + actions taken
vec_names = ['DG', 'YG', 'AG']
for i in range(5):
    vec_names.append('D' + str(i))
nVec = len(vec_names)
nTimesteps = 4
vecPad = 10
for instance in instances:
    print(instance)
    directory = instance_paths[instance]
    slideshow_path = directory + 'slideshow/'
    if not os.path.exists(slideshow_path):
        os.mkdir(slideshow_path)
    # read observations
    files = [file for file in os.listdir(directory) if 'observations' in file]
    observations = {}
    for file in files:
        observation_set = np.load(directory + file, allow_pickle=True)
        observations.update(observation_set)
    # plot slides and save as png files
    slides = []
    evaluations = [all_evaluations[instance][-1]]
    for evaluation, episodes in enumerate(evaluations):
        if 'train' in instance:
            episodes = episodes[0:10]
        for episode, states in enumerate(episodes):
            for step, state in enumerate(states):
                if step == 0:
                    continue
                # get state values
                transcribed_action = state['transcribed_action']
                done = state['done']
                if done:
                    termination_reason = state['termination_reason']
                observation_key = state['observation_component']
                # get observation and params
                if observation_key not in observations:
                    continue
                observation = observations[observation_key]
                # check if observation is multi, vec, or img
                if len(observation.shape) == 0:
                    obs_type = 'multi'
                    img_array = observation.item()['img']
                    vec_array = observation.item()['vec']
                elif len(observation.shape) == 1:
                    obs_type = 'vec'
                    nRows = 1
                    nCols = 84
                    empty_array = np.zeros((nRows, nCols*nTimesteps + nTimesteps - 1), dtype=np.int16)
                    vec_array = observation
                else:
                    obs_type = 'img'
                    img_array = observation
                # show image array
                if obs_type in ['multi', 'img']:
                    nBands = img_array.shape[0]
                    nRows = img_array.shape[1]
                    nCols = img_array.shape[2]
                    view_shape = (nRows, nBands * nCols + nBands - 1)
                    # view side by side
                    side_by_side = np.zeros(view_shape, dtype=np.int16)
                    side_by_side
                    for band in range(nBands):
                        side_by_side[0:nRows, (band*nCols+band):(band*nCols+band)+nCols] = img_array[band]
                    plt.plot([0, view_shape[1]], [21, 21], color='green')
                    plt.plot([0, view_shape[1]], [42, 42], color='green')
                    plt.plot([0, view_shape[1]], [63, 63], color='green')
                    
                    for i in range(nBands):
                        plt.plot([16 + i*nCols+i, 16 + i*nCols+i], [0, 84], color='green')
                        plt.plot([32 + i*nCols+i, 32 + i*nCols+i], [0, 84], color='green')
                        plt.plot([52 + i*nCols+i, 52 + i*nCols+i], [0, 84], color='green')
                        plt.plot([68 + i*nCols+i, 68 + i*nCols+i], [0, 84], color='green')
                    plt.imshow(side_by_side, cmap='gray')
                # show vector array
                if obs_type in ['multi', 'vec']:
                    if obs_type == 'vec':
                        plt.imshow(empty_array, cmap='gray')
                    x = 0
                    for time in range(nTimesteps):
                        y = nRows + vecPad
                        for i, name in enumerate(vec_names):
                            value = vec_array[time*nVec + i]
                            plt.text(x, y, name + ':' + str(round(value, 2)))
                            y += vecPad
                        x += nCols + 1
                plt.axis('off')
                title = 'Eps ' + str(episode) + '  Step ' + str(step) + '  Act ' + transcribed_action
                title += '' if not done else '  Term ' + termination_reason
                plt.title(title)
                #slide_name = title + '.png'
                #plt.savefig(slideshow_path + slide_name)
                #slides.append(slide_name)
                plt.show()      
                
    # make video from frames
    frames = []
    for slide_name in slides:
        frame_path = slideshow_path + slide_name
        frame = cv2.imread(frame_path)
        frame_shape = frame.shape
        frames.append(frame)
    video_path = slideshow_path + instance + '_slideshow.avi'
    video = cv2.VideoWriter(
                            filename = video_path, 
                            fourcc = 0,
                            fps = 1,
                            frameSize = (frame_shape[1], frame_shape[0]),
                            isColor = True,
                           )
    for frame in frames:
        video.write(frame)
    video.release()

In [ ]:
# play with reward function
import numpy as np
from matplotlib import pyplot as plt
def distance(d):
    #return -1*np.exp(-1*d)
    #return 1*(np.tanh(d) - 1)
    return .1*d
def steps(s):
    return -1
def resolution(c1, c2):
    return -2*c1#1*(np.exp(-*c1)-1) #+ np.exp(-.5*c2)-1 # two sensors to detect resolution
def goal(yes):
    return 100 if yes else 0
def collision(yes):
    return -100 if yes else 0

gamma = 0.99
start_distance = 100
qsc = []
qsS2 = []
qsS = []
qs = []
qs0 = []
qs1 = []
qs2 = []
qs3 = []
qs4 = []
steps_to_goal = []
max_steps = 1000
mean_steps = np.arange(1, 10, .1)
for mean_step in mean_steps:
    d = start_distance
    c = c1 = c2 = 4
    s = 0
    r = distance(d) + steps(s) + resolution(c1,c2) + goal(False) + collision(False)
    rc = distance(d) + steps(s+1) + resolution(c1,c2) + goal(False) + collision(True)
    qc = r+gamma*rc
    qsc.append(qc)
    q = q0 = q1 = q2 = q3 = q4 = qS = qS2 = r
    not_added = True
    while(True):
        s += 1
        #step = mean_step
        step = np.random.normal(mean_step, mean_step)
        d = d-step
        reached = d < 4
        r = distance(0) + steps(s) + resolution(0,0) + goal(False) + collision(False)
        r0 = distance(step) + steps(s) + resolution(0,0) + goal(reached) + collision(False)
        r1 = distance(step) + steps(s) + resolution(1,1) + goal(reached) + collision(False)
        r2 = distance(step) + steps(s) + resolution(2,2) + goal(reached) + collision(False)
        r3 = distance(step) + steps(s) + resolution(3,3) + goal(reached) + collision(False)
        r4 = distance(step) + steps(s) + resolution(4,4) + goal(reached) + collision(False)
        rS2 = distance(step) + steps(s) + resolution(0,0) + goal(False) + collision(False)
        q += r * gamma**s
        q0 += r0 * gamma**s
        q1 += r1 * gamma**s
        q2 += r2 * gamma**s
        q3 += r3 * gamma**s
        q4 += r4 * gamma**s
        qS2 += rS2 * gamma**s
        if reached and not_added:
            steps_to_goal.append(s)
            qs0.append(q0)
            qs1.append(q1)
            qs2.append(q2)
            qs3.append(q3)
            qs4.append(q4)
            qsS2.append(qS2)
            not_added = False
        if s > max_steps:
            break
    if not_added:
        steps_to_goal.append(s)
        qs0.append(q0)
        qs1.append(q1)
        qs2.append(q2)
        qs3.append(q3)
        qs4.append(q4)
        qsS2.append(qS2)
        not_added = False
    qs.append(q)
    s = 0
    d = start_distance
    while(True):
        s += 1
        step = np.random.normal(0, mean_step)
        d = d-step
        reached = d < 4
        rS = distance(step) + steps(s) + resolution(0,0) + goal(reached) + collision(False)
        qS += rS * gamma**s
        if reached:
            break
        if s > max_steps:
            break
    qsS.append(qS)
steps_to_goal = np.log10(steps_to_goal)
plt.plot(steps_to_goal, qsc, marker='.', label='Collision')
plt.plot(steps_to_goal, qs, marker='.', label='Do Nothing')
plt.plot(steps_to_goal, qs0, marker='.', label='Goal pwr=0')
plt.plot(steps_to_goal, qs1, marker='.', label='Goal pwr=1')
plt.plot(steps_to_goal, qs2, marker='.', label='Goal pwr=2')
plt.plot(steps_to_goal, qs3, marker='.', label='Goal pwr=3')
plt.plot(steps_to_goal, qs4, marker='.', label='Goal pwr=4')
plt.plot(steps_to_goal, qsS, marker='.', label='Steps pwr=0')
plt.plot(steps_to_goal, qsS2, marker='.', label='NG pwr=0')
#plt.xticks([i for i in range(1,4)])
plt.legend(bbox_to_anchor=(1,1))
plt.xlabel(f'log_10(# Steps to Goal)')
plt.ylabel('Q-Value')
plt.show()

In [ ]:
# play with reward function
plt.rcParams.update({'font.size': 22})
import numpy as np
from matplotlib import pyplot as plt
def distance(d):
    #return -1*np.exp(-1*d)
    return .1*(np.tanh(d))
    #return 0.1*d
def steps(s):
    return -1
def resolution(c1, c2):
    return -1*c1#1*(np.exp(-*c1)-1) #+ np.exp(-.5*c2)-1 # two sensors to detect resolution
def goal(yes):
    return 100 if yes else 0
def collision(yes):
    return -100 if yes else 0

gamma = 0.99
start_distance = 100
qsc = []
qsS2 = []
qsS3 = []
qsS4 = []
qsS5 = []
qsS6 = []
qsS = []
qs = []
qs0 = []
qs1 = []
qs2 = []
qs3 = []
qs4 = []
steps_to_goal = []
max_steps = 1000
mean_steps = np.arange(.1, 10, .1)
for mean_step in mean_steps:
    d = start_distance
    c = c1 = c2 = 4
    s = 0
    r = distance(d) + steps(s) + resolution(c1,c2) + goal(False) + collision(False)
    rc = distance(d) + steps(s+1) + resolution(c1,c2) + goal(False) + collision(True)
    qc = r+gamma*rc
    qsc.append(qc)
    q = q0 = q1 = q2 = q3 = q4 = qS = qS2 = qS3 = qS4 = qS5 = qS6 = r
    not_added = True
    while(True):
        s += 1
        #step = mean_step
        step = np.random.normal(mean_step, mean_step)
        d = d-step
        reached = d < 4
        r = distance(0) + steps(s) + resolution(0,0) + goal(False) + collision(False)
        r0 = distance(step) + steps(s) + resolution(0,0) + goal(reached) + collision(False)
        r1 = distance(step) + steps(s) + resolution(1,1) + goal(reached) + collision(False)
        r2 = distance(step) + steps(s) + resolution(2,2) + goal(reached) + collision(False)
        r3 = distance(step) + steps(s) + resolution(3,3) + goal(reached) + collision(False)
        r4 = distance(step) + steps(s) + resolution(4,4) + goal(reached) + collision(False)
        rS2 = distance(step) + steps(s) + resolution(0,0) + goal(False) + collision(False)
        rS3 = distance(step) + steps(s) + resolution(1,1) + goal(False) + collision(False)
        rS4 = distance(step) + steps(s) + resolution(2,2) + goal(False) + collision(False)
        rS5 = distance(step) + steps(s) + resolution(3,3) + goal(False) + collision(False)
        rS6 = distance(step) + steps(s) + resolution(4,4) + goal(False) + collision(False)
        q += r * gamma**s
        q0 += r0 * gamma**s
        q1 += r1 * gamma**s
        q2 += r2 * gamma**s
        q3 += r3 * gamma**s
        q4 += r4 * gamma**s
        qS2 += rS2 * gamma**s
        qS3 += rS3 * gamma**s
        qS4 += rS4 * gamma**s
        qS5 += rS5 * gamma**s
        qS6 += rS6 * gamma**s
        if reached and not_added:
            steps_to_goal.append(s)
            qs0.append(q0)
            qs1.append(q1)
            qs2.append(q2)
            qs3.append(q3)
            qs4.append(q4)
            qsS2.append(qS2)
            qsS3.append(qS3)
            qsS4.append(qS4)
            qsS5.append(qS5)
            qsS6.append(qS6)
            not_added = False
        if s > max_steps:
            break
    if not_added:
        steps_to_goal.append(s)
        qs0.append(q0)
        qs1.append(q1)
        qs2.append(q2)
        qs3.append(q3)
        qs4.append(q4)
        qsS2.append(qS2)
        qsS3.append(qS3)
        qsS4.append(qS4)
        qsS5.append(qS5)
        qsS6.append(qS6)
        not_added = False
    qs.append(q)
    s = 0
    d = start_distance
    while(True):
        s += 1
        step = np.random.normal(0, mean_step)
        d = d-step
        reached = d < 4
        rS = distance(step) + steps(s) + resolution(0,0) + goal(reached) + collision(False)
        qS += rS * gamma**s
        if reached:
            break
        if s > max_steps:
            break
    qsS.append(qS)

steps_to_goal = np.log10(steps_to_goal)
steps_to_goal2 = steps_to_goal.copy()
steps_to_goal2.sort()
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsc))], marker='.', label='Collision')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qs))], marker='.', label='Do Nothing')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qs0))], marker='.', label='Goal pwr=0')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qs1))], marker='.', label='Goal pwr=1')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qs2))], marker='.', label='Goal pwr=2')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qs3))], marker='.', label='Goal pwr=3')
#plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsc))]qs4, marker='.', label='Goal pwr=4')
#plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsc))]qsS, marker='.', label='Steps pwr=0')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsS2))], marker='.', label='No-Goal pwr=0')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsS3))], marker='.', label='No-Goal pwr=1')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsS4))], marker='.', label='No-Goal pwr=2')
plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsS5))], marker='.', label='No-Goal pwr=3')
#plt.plot(steps_to_goal2, [x for _, x in sorted(zip(steps_to_goal, qsc))]qsS6, marker='.', label='NG pwr=4')
#plt.xticks([i for i in range(1,4)])
plt.title('Random Walk')
#plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
#          fancybox=True, shadow=True, ncol=3)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel(r'$log_{10}$(# Steps to Goal)')
plt.ylabel('Q-Value')
#plt.yticks([-300, -200, -100, 0, 100])
plt.show()

In [ ]:
steps_to_goal